In [1]:
import sys
import os
import pandas as pd
from pandas.api.types import is_string_dtype
import math
import re
from tqdm import tqdm_notebook, tqdm
import j_utils.munging as mg
from lendingclub import config

rli = pd.read_feather(os.path.join(config.data_dir, 'raw_loan_info.fth'))
# Mangle grade and subgrade to make them useless since their meaning
# has changed so much over the years (As of 10/4/2019) LC doesn't
# have E,F,G grades
rli['grade'] = 0
# take a random sample of 1000 loans to work with
rli = rli.sample(20000, random_state=42)
dev_ids = rli['id']

In [6]:
rli['grade']

0          A
1          A
2          D
3          A
4          C
          ..
2507477    C
2507478    C
2507479    C
2507480    B
2507481    C
Name: grade, Length: 2507482, dtype: object

In [8]:
rli['sub_grade']

0          A5
1          A2
2          D2
3          A2
4          C4
           ..
2507477    C3
2507478    C3
2507479    C1
2507480    B3
2507481    C5
Name: sub_grade, Length: 2507482, dtype: object

In [5]:
for g, group in rli.groupby('grade'):
    print(pd.to_datetime(group['issue_d']).max())

2019-06-01 00:00:00
2019-06-01 00:00:00
2019-06-01 00:00:00
2019-06-01 00:00:00
2019-05-01 00:00:00
2019-06-01 00:00:00
2019-03-01 00:00:00


In [2]:
print(dev_ids.to_list())

[135846826, 126461631, 79581442, 124857628, 140938251, 128227825, 77350855, 1622294, 94750460, 40440155, 58460477, 122236264, 58310451, 108815122, 117272801, 58723912, 147098010, 7439256, 137750089, 146504821, 86101283, 144364154, 1501323, 90096681, 67848659, 57853116, 111569907, 63516265, 38589235, 92577636, 41288227, 67266298, 58722697, 133780807, 105413345, 44945334, 96835618, 125050100, 132023104, 128356051, 152254724, 134600581, 124044992, 40450344, 1096267, 69694504, 112061999, 132489437, 115096387, 75859145, 3369311, 145439445, 147859767, 137363962, 68192137, 66666852, 127340822, 61922528, 102576688, 56019293, 58311146, 136740571, 149434547, 6168240, 134782210, 151500235, 92331652, 124133601, 131108991, 109081138, 43580089, 120198017, 68845606, 93653072, 3675519, 153301478, 148709078, 3163681, 21490500, 137497179, 128298890, 105293137, 68617014, 116708031, 57034995, 126441771, 91843262, 147486222, 96879243, 149450336, 125302795, 121221062, 134878138, 50536945, 33311596, 92471041

In [3]:
import pickle

with open(os.path.join(config.data_dir,'dev_ids.pkl'), 'wb') as f:
    pickle.dump(dev_ids.to_list(), f)